# Ring Bus Substation Test

## Create a New Substation

### Import CIMantic Graphs Library and CIM profile

In [1]:
# Import transmission and distribution modeling classes
from cimgraph.models import FeederModel, NodeBreakerModel
from cimgraph.databases import ConnectionParameters, RDFlibConnection, BlazegraphConnection
import cimgraph.utils as utils

import importlib

cim_profile = 'cimhub_2023'
cim = importlib.import_module('cimgraph.data_profile.' + cim_profile)

params = ConnectionParameters(filename=None, cim_profile=cim_profile, iec61970_301=8)
connection = RDFlibConnection(params)

### Create New Main-and-Transfer Substation

In [2]:
from cimbuilder.substation_builder import RingBusSubstation

In [3]:
SubBuilder = RingBusSubstation(connection=connection, name="ring_sub", base_voltage=11500, total_sections=4)
substation = SubBuilder.substation

### Add IEEE 13 Feeder and IEEE 13 Assets Feeder to substation

In [4]:
# Import 13 bus model from XML file
ieee13_feeder = cim.Feeder(mRID = '49AD8E07-3BF9-A4E2-CB8F-C3722F837B62')
params = ConnectionParameters(filename='../test_models/IEEE13.xml', cim_profile=cim_profile, iec61970_301=8)
connection = RDFlibConnection(params)
ieee13_network = FeederModel(connection=connection, container=ieee13_feeder, distributed=False)


In [5]:

assets13_feeder = cim.Feeder(mRID = '5B816B93-7A5F-B64C-8460-47C17D6E4B0F')
params = ConnectionParameters(filename='../test_models/IEEE13_Assets.xml', cim_profile=cim_profile, iec61970_301=8)
connection = RDFlibConnection(params)
assets13_network = FeederModel(connection=connection, container=assets13_feeder, distributed=False)


In [6]:
SubBuilder.new_ring_bus_feeder(bus_number = 1, feeder=ieee13_feeder, feeder_network=ieee13_network)
SubBuilder.new_ring_bus_feeder(bus_number = 2, feeder=assets13_feeder, feeder_network=assets13_network)

In [7]:
SubBuilder.network.pprint(cim.Substation)

{
    "9fa3d4e9-9f04-44a7-aeed-b61d47c220fb": {
        "mRID": "9fa3d4e9-9f04-44a7-aeed-b61d47c220fb",
        "name": "ring_sub"
    }
}


In [8]:
SubBuilder.network.pprint(cim.Feeder)

{
    "49AD8E07-3BF9-A4E2-CB8F-C3722F837B62": {
        "mRID": "49AD8E07-3BF9-A4E2-CB8F-C3722F837B62",
        "name": "ieee13nodeckt",
        "Location": "8E4E3C92-0B7A-4F74-8FD2-CC10F74E452F",
        "ConnectivityNodes": "['673E896A-DCBF-4E43-9924-BEB31C5B6005', 'A8A25B50-3AE3-4A31-A18B-B3FA13397ED3', '421E99BE-A834-4809-B924-84D88F634A45', '2A6DC4DD-D3DC-434D-A187-D2C58A0A72C8', '7BEDDADD-0A14-429F-8601-9EA8B892CA6E', '63DFBEA0-CD06-4D2E-B956-DF9517BE057B', '30BE5988-DE57-4E0C-AB08-50D5A13D2C1B', '0124E881-B82D-4206-BBDF-37D585159872', 'C6256170-E6ED-4F91-8EBD-748090C3FDD5', '0A98A62D-7642-4F03-8317-A8605CBDBA37', '04984C4D-CC29-477A-9AF4-61AC7D74F16F', '0DCC57AF-F4FA-457D-BB24-2EFDA9865A1A', 'E5B2888B-B60D-4DA6-A4F7-17EB849D28B2', '6CB5E5CE-2CD0-40CC-A979-B4F9ED05E49B', '8E99F99D-FE8F-420B-AC49-0B52DF5362AB', 'DC889FA5-7B28-4273-A1D7-205BE3E0BFED', '94F822E0-7130-4205-8597-B47110BBEF4B', 'ADDB7A30-5A3C-4179-AF5D-5C9A7213B0E7', '8C58660F-C62C-4903-BE72-22F1255B1E62', '76D6D03C-96

### Save Substation to XML File

In [9]:
utils.write_xml(SubBuilder.network, '../test_output/ring_bus.xml')

## Round-Trip Test: Load and Read from Database

### Delete all old entries from database and load new models

In [10]:
# Connect to Blazegraph Database
from cimgraph.databases import BlazegraphConnection
params = ConnectionParameters(url = "http://localhost:8889/bigdata/namespace/kb/sparql", cim_profile=cim_profile, iec61970_301=8)
blazegraph = BlazegraphConnection(params)
blazegraph.execute('drop all')

/home/ande188/CIM-Builder/.venv/lib/python3.10/site-packages/SPARQLWrapper/Wrapper.py:1215: RuntimeWarning: unknown response content type 'text/html;charset=utf-8' returning raw response...
  warnings.warn(


b'<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN" "http://www.w3.org/TR/html4/loose.dtd"><html><head><meta http-equiv="Content-Type" content="text&#47;html;charset=UTF-8"><title>blazegraph&trade; by SYSTAP</title\n></head\n><body<p>totalElapsed=21ms, elapsed=20ms, connFlush=0ms, batchResolve=0, whereClause=0ms, deleteClause=0ms, insertClause=0ms</p\n><hr><p>COMMIT: totalElapsed=36ms, commitTime=1707881484433, mutationCount=6671</p\n></html\n>'

In [11]:
from cimloader.databases.blazegraph import BlazegraphConnection as BlazegraphLoader
params = ConnectionParameters(url = "http://localhost:8889/bigdata/namespace/kb/sparql", cim_profile=cim_profile, iec61970_301=8)
loader = BlazegraphLoader(params)

In [12]:
loader.upload_from_file(filename='../test_models/IEEE13.xml')
loader.upload_from_file(filename='../test_models/IEEE13_Assets.xml')
loader.upload_from_file(filename='../test_output/ring_bus.xml')

HTTP/1.1 100 Continue

HTTP/1.1 200 OK
Content-Type: application/xml;charset=iso-8859-1
Content-Length: 62
Server: Jetty(9.4.18.v20190429)

<?xml version="1.0"?><data modified="3392" milliseconds="21"/>HTTP/1.1 100 Continue

HTTP/1.1 200 OK
Content-Type: application/xml;charset=iso-8859-1
Content-Length: 62
Server: Jetty(9.4.18.v20190429)

<?xml version="1.0"?><data modified="2927" milliseconds="23"/>HTTP/1.1 100 Continue

HTTP/1.1 200 OK
Content-Type: application/xml;charset=iso-8859-1
Content-Length: 61
Server: Jetty(9.4.18.v20190429)

<?xml version="1.0"?><data modified="383" milliseconds="14"/>

### Create new NodeBreakerModel of Substation + Feeders

In [13]:
network = NodeBreakerModel(container=substation, connection=blazegraph, distributed = False)

In [14]:
# Print substation info
network.get_all_edges(cim.Substation)
network.pprint(cim.Substation)

{
    "9fa3d4e9-9f04-44a7-aeed-b61d47c220fb": {
        "mRID": "9fa3d4e9-9f04-44a7-aeed-b61d47c220fb",
        "name": "ring_sub",
        "ConnectivityNodes": "['827b2348-ba18-4685-af45-9c93635952a1', '9b8e8f69-8aa2-4cc5-9fea-0c329a7a7ff8', 'beb6080c-6405-477f-910a-ffa6071bd523', 'ceb526bf-38b8-4b76-a55f-f2c9d56b8508', 'f7bdaf97-7fa9-4504-8d33-8e50b76e5a81', '0ede1480-d606-46d1-b1bb-ba8bd79c0cf8', '10aacc67-757b-4f68-a255-1f8e9eebd2bc', '1ba465df-3786-4627-8214-e9a3870dd2f1', '311b50b1-6767-4197-974c-c85bc12a4cc8', '383c62d5-234b-45df-b87e-8fdc9d2ab96c', '4b4e7876-1a7e-40f1-86fe-8a04f6be5324', '7daf9c7d-ea18-4da4-a101-6f3e3313591f']",
        "Equipments": "['927a5833-762d-478c-83e8-b6fa6aa1e23c', '9310f8e5-d870-447f-88aa-f9eaa919b872', 'cc32a428-1eda-4a0c-8c14-7fa56ce9442a', 'd3c675ae-d0c2-4d02-9c09-9e1c9761dba4', 'd9e4d822-1b9b-4a18-927c-af0c221601a9', 'ee4010fa-02b3-4cea-a7e2-2c3fd2c30089', 'f1b00009-1bbf-4233-b9dc-1d26dab2fe2e', '1499578e-33e3-4907-9cca-0b2f9d1abff8', '2c5a9084-7

In [15]:
# Print feeder info
network.get_all_edges(cim.Feeder)
network.pprint(cim.Feeder)

{
    "49AD8E07-3BF9-A4E2-CB8F-C3722F837B62": {
        "mRID": "49AD8E07-3BF9-A4E2-CB8F-C3722F837B62",
        "name": "ieee13nodeckt",
        "Location": "8E4E3C92-0B7A-4F74-8FD2-CC10F74E452F",
        "ConnectivityNodes": "['2A6DC4DD-D3DC-434D-A187-D2C58A0A72C8', '30BE5988-DE57-4E0C-AB08-50D5A13D2C1B', '421E99BE-A834-4809-B924-84D88F634A45', '63DFBEA0-CD06-4D2E-B956-DF9517BE057B', '673E896A-DCBF-4E43-9924-BEB31C5B6005', '6CB5E5CE-2CD0-40CC-A979-B4F9ED05E49B', '76D6D03C-967C-4E12-836D-A98DF3552BC7', '7BEDDADD-0A14-429F-8601-9EA8B892CA6E', '8C58660F-C62C-4903-BE72-22F1255B1E62', '8E99F99D-FE8F-420B-AC49-0B52DF5362AB', '94F822E0-7130-4205-8597-B47110BBEF4B', 'A8A25B50-3AE3-4A31-A18B-B3FA13397ED3', 'ADDB7A30-5A3C-4179-AF5D-5C9A7213B0E7', 'C6256170-E6ED-4F91-8EBD-748090C3FDD5', 'DBFA8EFA-B397-40DA-9642-60E8DE4B3471', 'DC889FA5-7B28-4273-A1D7-205BE3E0BFED', 'E5B2888B-B60D-4DA6-A4F7-17EB849D28B2', '0124E881-B82D-4206-BBDF-37D585159872', '04984C4D-CC29-477A-9AF4-61AC7D74F16F', '0A98A62D-76

In [16]:
# Print total load served by substation from both feeders
total_load = 0
network.get_all_edges(cim.EnergyConsumer)
for load in network.graph[cim.EnergyConsumer].values():
    total_load = total_load + float(load.p)

print(f'total load is {total_load/1000} kW')

total load is 6937.0 kW


### Merge substation and feeders into a single XML file 

In [17]:
utils.get_all_data(network)

In [18]:
utils.write_xml(network, '../test_output/ring_bus_and_feeders.xml')


![ring-bus](./ring_bus.png)